In [1]:
import pandas as pd
import numpy as np
FAC = pd.read_csv('lv2-2405.csv')
print(FAC.shape)
FAC.info()

(1006, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006 entries, 0 to 1005
Data columns (total 7 columns):
PRD_ID       1006 non-null int64
ROT_SPD      991 non-null float64
TRQ_PWR      1006 non-null int64
DUR          1006 non-null int64
NS           1006 non-null int64
TEMP_DIFF    1006 non-null float64
FAIL_YN      1006 non-null int64
dtypes: float64(2), int64(5)
memory usage: 55.1 KB


### Question 1

In [2]:
# 1-1
FAC0 = FAC[FAC['FAIL_YN']==0].copy()
FAC1 = FAC[FAC['FAIL_YN']==1].copy()
print(FAC0.shape, FAC1.shape)

(965, 7) (41, 7)


In [3]:
# 1-2
cvval = ['TRQ_PWR','DUR','NS','TEMP_DIFF']
CV0 = FAC0[cvval].apply(lambda x: x.std() / x.mean())
CV1 = FAC1[cvval].apply(lambda x: x.std() / x.mean())
print(CV0)
print(CV1)

TRQ_PWR      0.231706
DUR          0.566948
NS           0.116885
TEMP_DIFF    0.092719
dtype: float64
TRQ_PWR      0.433642
DUR          0.552811
NS           0.128271
TEMP_DIFF    0.130327
dtype: float64


In [4]:
# 1-3
CV_diff = abs(CV0-CV1)
print(CV_diff)
diff_max = max(CV_diff)
print('max:',diff_max)

TRQ_PWR      0.201936
DUR          0.014137
NS           0.011386
TEMP_DIFF    0.037607
dtype: float64
max: 0.20193552889678829


In [5]:
# Answer 1
ans24 = round(diff_max, 3)
ans24

0.202

### Question 2

In [6]:
# 2-1
DF5 = FAC.dropna(subset=['ROT_SPD']).copy()
print(DF5.shape)

(991, 7)


In [7]:
# 2-2
from sklearn.preprocessing import StandardScaler
SVAR = ['ROT_SPD','TRQ_PWR','DUR','NS','TEMP_DIFF']
DF5s = StandardScaler().fit_transform(DF5[SVAR])
SVARS = [var + '_S' for var in SVAR]
DF5s = pd.DataFrame(DF5s, columns=SVARS)
DF5s.head()

,ROT_SPD_S,TRQ_PWR_S,DUR_S,NS_S,TEMP_DIFF_S
0,-0.511206,0.893629,0.867364,-0.771185,-0.001684
1,0.760403,-0.916769,-0.502271,-1.248242,0.109599
2,-0.242934,0.089008,0.628474,-0.850695,0.109599
3,-0.210741,0.692474,1.408848,0.262437,1.333715
4,-0.865325,0.591896,1.584034,-0.055600,-1.114517


In [8]:
# 2-3
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3, random_state=1234, init='random', n_init=5).fit(DF5s)
DF5['clst'] = km.labels_

In [9]:
# 2-4
DF5o = DF5.groupby(['clst']).agg(FAIL_RT=('FAIL_YN', 'mean'),
                                 N = ('FAIL_YN', 'count')).reset_index()
DF5o.FAIL_RT = 100*DF5o.FAIL_RT
print(DF5o)
FAIL_RT_DIFF = abs(max(DF5o.FAIL_RT) - min(DF5o.FAIL_RT))
print('Diff:',FAIL_RT_DIFF)

   clst   FAIL_RT    N
0     0  1.459854  411
1     1  6.469003  371
2     2  5.263158  209
Diff: 5.00914868081925


In [10]:
# Answer 2
ans25 = round(FAIL_RT_DIFF, 2)
ans25

5.01

### Question 3

In [11]:
# 3-1
DF6 = FAC.copy()
median_value = FAC['ROT_SPD'].median()
DF6['ROT_SPD'].fillna(median_value, inplace=True)

In [12]:
# 3-2
TRAIN6 = DF6[DF6['PRD_ID']%3!=0].copy()
TEST6 = DF6[DF6['PRD_ID']%3==0].copy()
print(TRAIN6.shape, TEST6.shape)

(671, 7) (335, 7)


In [13]:
# 3-3
from sklearn.tree import DecisionTreeClassifier
VAR6 = ['ROT_SPD','TRQ_PWR','DUR','NS','TEMP_DIFF']
DT3 = DecisionTreeClassifier(random_state=1234, max_depth=3, min_samples_split=3).fit(TRAIN6[VAR6], TRAIN6.FAIL_YN)
DT5 = DecisionTreeClassifier(random_state=1234, max_depth=5, min_samples_split=3).fit(TRAIN6[VAR6], TRAIN6.FAIL_YN)
DT7 = DecisionTreeClassifier(random_state=1234, max_depth=7, min_samples_split=3).fit(TRAIN6[VAR6], TRAIN6.FAIL_YN)
DTS = [DT3, DT5, DT7]

In [14]:
# 3-4
TRAIN6['PRED3'] = DT3.predict(TRAIN6[VAR6])
TRAIN6['PRED5'] = DT5.predict(TRAIN6[VAR6])
TRAIN6['PRED7'] = DT7.predict(TRAIN6[VAR6])
RECALLS = [np.mean(TRAIN6[TRAIN6['FAIL_YN']==1].PRED3==1),
           np.mean(TRAIN6[TRAIN6['FAIL_YN']==1].PRED5==1),
           np.mean(TRAIN6[TRAIN6['FAIL_YN']==1].PRED7==1)]
print(RECALLS)
DT_FIN = DTS[np.argmax(RECALLS)]
print(DT_FIN)

[0.37037037037037035, 0.6296296296296297, 0.7407407407407407]
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=1234, splitter='best')


In [15]:
# 3-5
TEST6['PRED'] = DT_FIN.predict(TEST6[VAR6])
RECALL6 = np.mean(TEST6[TEST6['FAIL_YN']==1].PRED==1)
RECALL6

0.6428571428571429

In [16]:
# Answer 3
ans26 = round(RECALL6, 3)
ans26

0.643

## Summary

In [17]:
print('ans24:',ans24)
print('ans25:',ans25)
print('ans26:',ans26)

ans24: 0.202
ans25: 5.01
ans26: 0.643
